In [3]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0971,  0.1985,  0.0569, -0.1455, -0.0634,  0.1430,  0.1671, -0.0604,
          0.0626,  0.0082],
        [-0.0295,  0.0499,  0.0332, -0.0407, -0.1907,  0.1860,  0.2307, -0.0100,
          0.0456, -0.1260]], grad_fn=<AddmmBackward0>)

### 5.1.1 自定义块

In [4]:
class MLP(nn.Module):
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        """
        定义前向传播函数。

        参数:
        - X: 输入数据，形状为 (batch_size, input_size)

        返回:
        - output: 经过前向传播后的输出数据，形状为 (batch_size, output_size)
        """
        # 首先通过隐藏层 self.hidden 对输入 X 进行线性变换
        hidden_output = self.hidden(X)
        # 然后对隐藏层的输出应用 ReLU 激活函数
        activated_output = F.relu(hidden_output)
        # 最后通过输出层 self.out 对激活后的输出进行线性变换，得到最终的输出
        output = self.out(activated_output)
        return output


In [5]:
net = MLP()
net(X)

tensor([[-0.1048,  0.1119,  0.0077,  0.0383, -0.2591,  0.0070, -0.1402,  0.1203,
         -0.0630, -0.0689],
        [-0.0914, -0.0065, -0.0427, -0.0815, -0.1343, -0.0185, -0.1663,  0.1132,
         -0.0631, -0.0194]], grad_fn=<AddmmBackward0>)

### 5.1.2 顺序块

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        """
        初始化MySequential类的实例。

        参数:
        *args: 可变数量的模块，将按照顺序添加到MySequential实例中。
        """
        super().__init__()
        # 遍历传入的模块列表
        for idx, module in enumerate(args):
            # 将每个模块添加到MySequential实例中，并使用索引作为模块的名称
            self._modules[str(idx)] = module

    def forward(self, X):
        """
        前向传播方法。

        参数:
        X: 输入数据。

        返回:
        经过所有模块处理后的输出数据。
        """
        # 遍历MySequential实例中的所有模块
        for block in self._modules.values():
            # 对输入数据进行前向传播，并将输出
            X = block(X)
        return X
            

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1671, -0.1302,  0.2239,  0.0965,  0.4869, -0.0497,  0.2815, -0.0545,
         -0.2662, -0.1040],
        [-0.0644, -0.1000,  0.1680,  0.1709,  0.2063, -0.1077,  0.1011, -0.0463,
         -0.1381, -0.0226]], grad_fn=<AddmmBackward0>)

### 5.1.3. 在前向传播函数中执行代码

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)        

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [17]:
net = FixedHiddenMLP()
net(X)

tensor(0.0339, grad_fn=<SumBackward0>)

In [18]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

In [20]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0198, grad_fn=<SumBackward0>)

In [ ]:
# Practice Problem 2
#实现一个块，它以两个块为参数，例如net1和net2，并返回前向传播中两个网络的串联输出。这也被称为平行块。
import torch
from torch import nn
from torch.nn import functional as F

class Parallel(nn.Module):
    def __init__(self, net1, net2):
        super().__init__()
        self.net1, self.net2 = net1, net2
    def forward(self, X):
        return torch.cat((self.net1(X), self.net2(X)), 1)
# Practice Problem 3
# 假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。
def block1():
    return nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=3, padding=1), nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net